In [9]:
import os
import pandas as pd
from ast import literal_eval
from bs4 import BeautifulSoup
import requests
import urllib.request
import json

Role: https://www.gov.uk/api/content/government/ministers/attorney-general

People: https://www.gov.uk/api/content/government/people/matthew-hancock and list of all people: https://www.gov.uk/api/content/government/people

In [2]:
DATA_DIR = os.getenv("DATA_DIR")

In [17]:
def extract_html_links(text):
    """
    Grab any GOV.UK domain-specific links from page text.
    :param text: Text within a details sub-section, refer to filtered for keys.
    :return: list of links
    """
    links = []
#     try:
    soup = BeautifulSoup(text, "html.parser")
    links = [link.get('href') for link in soup.findAll('a', href=True)]
#     except Exception:
#         print("error")
    return [l.replace("https://www.gov.uk/", "/") for l in links
            if l.startswith("/government/people")]

In [ ]:
"https://www.gov.uk/government/people"

In [ ]:
url = requests.get("https://www.gov.uk/government/people")
htmltext = url.text

In [21]:
int("https://www.gov.uk/government/people?page=2".split("=")[-1])

2

In [31]:
all_people = []
ind = 1
next_page = 1
while ind == next_page:
    target = "https://www.gov.uk/government/people?page={}".format(ind)
    url = requests.get(target)
    link_list = extract_html_links(url.text)
    all_people.extend(link_list[0:-1])
    ind+=1
    next_page = int(link_list[-1].split("=")[-1])
    print(ind)
    print(next_page)

2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
69


In [36]:
with open(os.path.join(DATA_DIR, "people_urls.csv"), "w") as write:
    for p in all_people:
        write.write(p+"\n")